# Exploring calculate and equilibrium xarray Datasets

## xarray Datasets

Results returned from calling <span style="background-color: #f0f0f0; padding: 2px;">calculate</span> or <span style="background-color: #f0f0f0; padding: 2px;">equilibrium</span> in pycalphad are xarray Datasets. An xarray Dataset is a data structure that represents N-dimensional tabular data. It is an N-dimensional analog to the Pandas DataFrame.

This notebook will walk through the structure of xarray Datasets in pycalphad and some basics of using them. For more in-depth tutorials and documentation on using xarray Datasets and DataArray’s fully, see the xarray [xarray documentation](https://docs.xarray.dev/en/stable/index.html)

## Dataset structure

Each Dataset stores the conditions that properties are calculated at and the values of the properties as a function of the different conditions. There are three key terms:

<span style="background-color: #f0f0f0; padding: 2px;">Dimensions</span>: these are the conditions that are calculated over, e.g. pressure (P) and temperature (T). They are essentially labels.

<span style="background-color: #f0f0f0; padding: 2px;">Coordinates</span>: these are the actual values that are taken on by the dimensions.

<span style="background-color: #f0f0f0; padding: 2px;">Data variables</span>: these are the properties calculated by pycalphad, such as the Gibbs energy, mixing energy, composition, etc.

### <span style="background-color: #f0f0f0; padding: 2px;">calculate()</span>  results



<span style="background-color: #f0f0f0; padding: 2px;">calculate</span> is used to sample properties of a single phase. There are five dimensions/coordinates:

<span style="background-color: #f0f0f0; padding: 2px;">P</span>: pressures (in Pa).

<span style="background-color: #f0f0f0; padding: 2px;">T</span>: temperatures (in K).

<span style="background-color: #f0f0f0; padding: 2px;">component</span>: the string names of the components in the system

<span style="background-color: #f0f0f0; padding: 2px;">internal_dof</span>: The internal_dof (internal degrees of freedom) is the index of the site in any phase’s site fraction array. Below the FCC_A1 phase has the sublattice model (AL, ZN) and thus the internal_dof are integers 0 and 1 referring to the AL site (index 0) and the ZN site (index 1).

<span style="background-color: #f0f0f0; padding: 2px;">points</span>: By default, the calculate function samples points over all of the internal degrees of freedom. Each coordinate point simply represents the index is a list of all configurations of the internal_dof sampled. There is no underlying physical meaning or order.

There are also at least four Data variables:

<span style="background-color: #f0f0f0; padding: 2px;">phase</span>: The string name of the phase. For <span style="background-color: #f0f0f0; padding: 2px;">calculate</span>, this will always be the phase name passed.

<span style="background-color: #f0f0f0; padding: 2px;">X</span>: The composition of each component in mole fraction as a function of the temperature, pressure, and the index of the points (there is one composition for each point).

<span style="background-color: #f0f0f0; padding: 2px;">Y</span>: The site fraction of each index in the internal_dof array for the given temperature, pressure and point.

<span style="background-color: #f0f0f0; padding: 2px;">output</span>: “output” is always whatever property is calculated by the output keyword passed to <span style="background-color: #f0f0f0; padding: 2px;">calculate</span>. The default is the molar Gibbs energy, GM.

## Importing modules

In [1]:
%matplotlib inline
from pycalphad import Database, calculate, equilibrium, variables as v

### Defining file paths for program and database

In [2]:
# Set the file path
tdb_diretory = r'C:\PythonCode\Environments\Calphad\pycalphad\pycalphad codes\1 - databases\\'

### Load database

In [3]:
tdb = Database(tdb_diretory + 'COST507_Sundman.tdb') #Aluminum database

### Load phases, elements, etc to consider

In [6]:
comps = ['AL', 'SI', 'VA']

### Carry out calculation and print output

In [7]:
calc_result = calculate(tdb, comps, 'FCC_A1', P=101325, T=[500, 1000])
print(calc_result)
#it prints the representation of the dataset object

<xarray.Dataset> Size: 576kB
Dimensions:    (N: 1, P: 1, T: 2, points: 4002, component: 2, internal_dof: 3)
Coordinates:
  * component  (component) <U2 16B 'AL' 'SI'
  * N          (N) float64 8B 1.0
  * P          (P) float64 8B 1.013e+05
  * T          (T) float64 16B 500.0 1e+03
Dimensions without coordinates: points, internal_dof
Data variables:
    X          (N, P, T, points, component) float64 128kB 1.0 1e-14 ... 0.2561
    Phase      (N, P, T, points) <U6 192kB 'FCC_A1' 'FCC_A1' ... 'FCC_A1'
    Y          (N, P, T, points, internal_dof) float64 192kB 1.0 1e-14 ... 1.0
    GM         (N, P, T, points) float64 64kB -1.558e+04 ... -3.689e+04
Attributes:
    phase_indices:  {'FCC_A1': slice(0, np.int64(4002), None)}


We can manipulate this by selecting data by value (of a coordinate) using sel or index (of a coordinate) using isel similar to a Pandas array. Below we get the site fraction of ZN (internal_dof index of 1 selected by index) at 1000K (selected by value) for the 50th point (selected by index).

The results of selecting over Data variables gives an xarray DataArray which is useful for plotting or performing computations on [(see DataArrays vs Datasets)](https://docs.xarray.dev/en/stable/user-guide/data-structures.html)

In [8]:
# .isel function allow us to select by index
# .sel function allow us to select by coordinate value

#For this example, in the .isel with points selected to 49, 
#it means give me thre result for which point is at the index being 49 
# with .sel, it gives the value at T=1000

print(calc_result.Y.isel(internal_dof=1, points=49).sel(T=1000))

<xarray.DataArray 'Y' (N: 1, P: 1)> Size: 8B
array([[0.97648824]])
Coordinates:
  * N        (N) float64 8B 1.0
  * P        (P) float64 8B 1.013e+05
    T        float64 8B 1e+03


accessing the <span style="background-color: #f0f0f0; padding: 2px;">values</span> attribute on any on any DataArray returns the multidimensional NumPy array



In [9]:
print(calc_result.X.values)

[[[[[1.00000000e+00 1.00000000e-14]
    [1.00000000e-14 1.00000000e+00]
    [1.00000000e-14 1.00000000e+00]
    ...
    [1.56995650e-01 8.43004350e-01]
    [1.12072782e-01 8.87927218e-01]
    [7.43933641e-01 2.56066359e-01]]

   [[1.00000000e+00 1.00000000e-14]
    [1.00000000e-14 1.00000000e+00]
    [1.00000000e-14 1.00000000e+00]
    ...
    [1.56995650e-01 8.43004350e-01]
    [1.12072782e-01 8.87927218e-01]
    [7.43933641e-01 2.56066359e-01]]]]]


# Equilibrium () results

<span style="background-color: #f0f0f0; padding: 2px;">P</span>: pressures (in Pa).

<span style="background-color: #f0f0f0; padding: 2px;">T</span>: temperatures (in K).

<span style="background-color: #f0f0f0; padding: 2px;">component</span>: (Same as calculate) The string names of the components in the system.

<span style="background-color: #f0f0f0; padding: 2px;">internal_dof</span>: (Same as calculate, except it will be the longest possible internal_dof for all phases) The internal_dof (internal degrees of freedom) is the index of the site in any phase’s site fraction array. Below the FCC_A1 phase has the sublattice model (AL, ZN) and thus the internal_dof are integers 0 and 1 referring to the AL site (index 0) and the ZN site (index 1).

<span style="background-color: #f0f0f0; padding: 2px;">X_ZN</span>: This is the composition of the species that was passed into the conditions array. Since we passed <span style="background-color: #f0f0f0; padding: 2px;">v.X('ZN')</span> to the conditions dictionary, this is <span style="background-color: #f0f0f0; padding: 2px;">X_ZN</span>.

<span style="background-color: #f0f0f0; padding: 2px;">vertex</span>: The vertex is the index of the phase in equilibrium. The vertex has no inherent physical meaning. There will automatically be enough to describe the number of phases present in any equilibria calculated, implying that vertex can never be large enough to invalidate Gibbs phase rule.

There are also at least six Data variables:

<span style="background-color: #f0f0f0; padding: 2px;">phase</span>: The string name of the phase in equilibrium at the conditions. There are as many as <span style="background-color: #f0f0f0; padding: 2px;">len(vertex)</span> phases. Any time there are fewer phases in equilibrium than the indices described by <span style="background-color: #f0f0f0; padding: 2px;">vertex</span>, the values of phase are paded by <span style="background-color: #f0f0f0; padding: 2px;">''</span>, e.g. for a single phase region for <span style="background-color: #f0f0f0; padding: 2px;">FCC_A1</span>, the values of Phase will be <span style="background-color: #f0f0f0; padding: 2px;">['FCC_A1', '']</span>. When more than one phase is present, it is important to note that they are not necessarily sorted.

<span style="background-color: #f0f0f0; padding: 2px;">NP</span>: Phase fraction of each phase in equilibrium. When there is no other equilibrium phase (e.g. single phase ['FCC_A1', '']) then the value of NP will be <span style="background-color: #f0f0f0; padding: 2px;">nan</span> for the absence of a phase, rather than 0.

<span style="background-color: #f0f0f0; padding: 2px;">MU</span>: The chemical potentials of each component for the conditions calculated.

<span style="background-color: #f0f0f0; padding: 2px;">X</span>: The equilibrium composition of each element in each phase for the calculated conditions.

<span style="background-color: #f0f0f0; padding: 2px;">Y</span>: The equilibrium site fraction of each site in each phase for the calculated conditions.

<span style="background-color: #f0f0f0; padding: 2px;">GM</span>: Same as <span style="background-color: #f0f0f0; padding: 2px;">output</span> for <span style="background-color: #f0f0f0; padding: 2px;">calculate</span>. It is always reported no matter the value of output.

<span style="background-color: #f0f0f0; padding: 2px;">output</span>: (optional) “output” is always whatever <span style="background-color: #f0f0f0; padding: 2px;">equilibrium</span> property is calculated by the output keyword passed to equilibrium. Unlike <span style="background-color: #f0f0f0; padding: 2px;">calculate</span>, this will be in addition to the <span style="background-color: #f0f0f0; padding: 2px;">GM</span> because <span style="background-color: #f0f0f0; padding: 2px;">GM</span> is always reported. <span style="background-color: #f0f0f0; padding: 2px;">HM</span> is molar enthalpy

In [14]:
# phases = ['LIQUID', 'FCC_A1', 'HCP_A3']
phases = list(tdb.phases.keys())
eq_result = equilibrium(tdb, comps , phases, {v.X('SI'):(0,1,0.05), v.T: (500, 1000, 100), v.P:101325}, output='HM')
print(eq_result)

<xarray.Dataset> Size: 32kB
Dimensions:    (N: 1, P: 1, T: 5, X_SI: 20, vertex: 3, component: 2,
                internal_dof: 4)
Coordinates:
  * N          (N) float64 8B 1.0
  * P          (P) float64 8B 1.013e+05
  * T          (T) float64 40B 500.0 600.0 700.0 800.0 900.0
  * X_SI       (X_SI) float64 160B 1e-10 0.05 0.1 0.15 0.2 ... 0.8 0.85 0.9 0.95
  * vertex     (vertex) int64 24B 0 1 2
  * component  (component) <U2 16B 'AL' 'SI'
Dimensions without coordinates: internal_dof
Data variables:
    NP         (N, P, T, X_SI, vertex) float64 2kB 1.0 nan nan ... 0.9409 nan
    GM         (N, P, T, X_SI) float64 800B -1.558e+04 -1.533e+04 ... -2.635e+04
    MU         (N, P, T, X_SI, component) float64 2kB -1.558e+04 ... -2.578e+04
    X          (N, P, T, X_SI, vertex, component) float64 5kB 1.0 1e-10 ... nan
    Y          (N, P, T, X_SI, vertex, internal_dof) float64 10kB 1.0 ... nan
    Phase      (N, P, T, X_SI, vertex) <U10 12kB 'FCC_A1' '' ... 'DIAMOND_A4' ''
    HM         (N

A common operation might be to find the phase fractions of the HCP_A3 phase as a function of composition for T=800.

However, the only way we can access the values of the phase fraction is by either the indices or values of the coordinates, we would have to know which index the HCP_A3 phase is in before hand to use the sel or isel commands.

Since we do not know this, we can do what is called [masking](https://docs.xarray.dev/en/stable/user-guide/indexing.html#masking-with-where) to find the data values that match a condition (the Phase is FCC_A1):




In [15]:
#This only prints the phase fraction of FCC_A1 under the .sel conditions. If no conditions are met, the output is nan
print(eq_result.NP.where(eq_result.Phase=='FCC_A1').sel(P=101325, T=800))

<xarray.DataArray 'NP' (N: 1, X_SI: 20, vertex: 3)> Size: 480B
array([[[1.        ,        nan,        nan],
        [0.96034898,        nan,        nan],
        [0.90980427,        nan,        nan],
        [0.85925957,        nan,        nan],
        [0.80871487,        nan,        nan],
        [0.75817017,        nan,        nan],
        [0.70762546,        nan,        nan],
        [0.65708076,        nan,        nan],
        [0.60653606,        nan,        nan],
        [0.55599136,        nan,        nan],
        [0.50544665,        nan,        nan],
        [0.45490195,        nan,        nan],
        [0.40435725,        nan,        nan],
        [0.35381255,        nan,        nan],
        [0.30326784,        nan,        nan],
        [0.25272314,        nan,        nan],
        [0.20217844,        nan,        nan],
        [0.15163374,        nan,        nan],
        [0.10108903,        nan,        nan],
        [0.05054433,        nan,        nan]]])
Coordinates:
  

This is to add gray background to words/text


<span style="background-color: #f0f0f0; padding: 2px;">phase</span>: 
   

https://www.youtube.com/watch?v=S-iOcrQlsnU